#### Загрузка данных

In [1]:
from pathlib import Path
import requests

In [2]:
data_dir = Path.cwd() / "data"
serb_train = data_dir / "sr_set-ud-train.conllu"

if not data_dir.exists():
    data_dir.mkdir()

if not serb_train.exists():
    
    url = r"https://raw.githubusercontent.com/UniversalDependencies/UD_Serbian-SET/refs/heads/master/sr_set-ud-train.conllu"

    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()

    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")

    else:
        with open(serb_train, "w", encoding="utf-8") as file:
            file.write(response.text)


### Подготовка данных

In [3]:
from nltk.parse import DependencyGraph
import warnings
warnings.filterwarnings('ignore')

In [4]:
with open(serb_train, "r", encoding="utf-8") as file:
    data = file.read()

In [5]:
sents = [sent for sent in data.split("\n\n") if sent.strip()]

In [6]:
len(sents)

3328

In [7]:
raw_texts = [
    "\n".join(
        filter(
            lambda line: line.strip().startswith("# text ="),
            sent.split("\n")
        )
    )[9:]
    for sent
    in sents
]

In [8]:
len(raw_texts)

3328

In [9]:
sents = [
    "\n".join(
        filter(
            lambda line: line.strip()[0] != "#",
            sent.split("\n")
        )
    )
    for sent
    in sents
]


In [10]:
trees = [
    DependencyGraph(sent)
    for sent
    in sents
]

In [11]:
len(trees)

3328

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("./data/lemmas2aspect.csv", sep="\t")
df.head()

,verb,cyrillic,count,aspect
0,baviti,бавити,9,imp
1,bacati,бацати,6,imp
2,birati,бирати,6,imp
3,biti,бити,5,both
4,blokirati,блокирати,10,imp


In [14]:
class Node:
    def __init__(self, tree_id: int, node_id: int) -> None:
        self.tree_id = tree_id
        self.node_id = node_id

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(tree={self.tree_id}, node={self.node_id})"


In [15]:
";".join("")

''

In [16]:
def getwnt(tree, id_):
    return tree.nodes[id_]["word"] + "," + tree.nodes[id_]["ctag"] + "," + tree.nodes[id_]["feats"]

In [17]:

data = []

for i, tree in enumerate(trees):
    for key in tree.nodes:
        node = tree.nodes[key]
        ctag = node.get("ctag", None)
        if ctag == "VERB" and node["lemma"] in df["verb"].values:

            nsubj = node['deps']["nsubj"]
            obj = node['deps']["obj"]
            iobj = node['deps']["iobj"]
            if iobj:
                print(node["word"])
                print(iobj)
                print(raw_texts[i])
            obl = node['deps']["obl"]


            new_row = {
                "lemma"  :  node["lemma"]  ,
                "word"   :  node["word"]  ,
                "aspect" :  df.loc[df["verb"] == node["lemma"]].values[0][-1]  ,
                "feats"  :  node["feats"]  ,
                "nsubj"  :  ";".join([getwnt(tree, ss) for ss in nsubj]), 
                "obj"    :  ";".join([getwnt(tree, ss) for ss in obj]), 
                "iobj"   :  ";".join([getwnt(tree, ss) for ss in iobj]), 
                "obl"    :  ";".join([getwnt(tree, ss) for ss in obl]), 
                "rel"    :  node["rel"]  ,
                "text"   : raw_texts[i]   ,
            }

            data.append(new_row)


omogućava
[5]
Dizajn novog muzeja omogućava posetiocima da proučavaju mermere Partenona dok istovremeno gledaju njihovu originalnu lokaciju - sam Akropolj.


In [18]:
assdf = pd.DataFrame(data)
assdf.head()

,lemma,word,aspect,feats,nsubj,obj,iobj,obl,rel,text
0,tvrditi,tvrde,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"Neki,DET,Case=Nom|Gender=Masc|Number=Plur|Pron...",,,,root,Neki tvrde da je presuda Veliji Ramkovskom nap...
1,tvrditi,tvrde,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"drugi,ADJ,Case=Nom|Degree=Pos|Gender=Masc|Numb...",,,,conj,Neki tvrde da je presuda Veliji Ramkovskom nap...
2,govoriti,govore,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"dokazi,NOUN,Case=Nom|Gender=Masc|Number=Plur",,,,ccomp,Neki tvrde da je presuda Veliji Ramkovskom nap...
3,moći,mogla,imp,Gender=Fem|Number=Sing|Tense=Past|VerbForm=Par...,"presuda,NOUN,Case=Nom|Gender=Fem|Number=Sing",,,,root,Krivična presuda i zatvorska kazna medijskom m...
4,kritikovati,kritikovao,both,Gender=Masc|Number=Sing|Tense=Past|VerbForm=Pa...,"koji,DET,Case=Nom|Gender=Masc|Number=Sing|Pron...","vladu,NOUN,Case=Acc|Gender=Fem|Number=Sing",,,acl,Krivična presuda i zatvorska kazna medijskom m...


In [19]:
import csv

In [20]:
assdf.sort_values(["lemma"]).to_csv("./forms_to_sents.csv", sep="\t", quoting=csv.QUOTE_NONE)

In [114]:
assdf.to_csv("./forms_to_sents.csv", sep="\t")

In [11]:
from collections import Counter

In [12]:
verbs_counts = Counter(
    {
        lemma: sum(
            [
                len(verbs[lemma][form])
                for
                    form
                in
                    verbs[lemma]
            ]
        )
        for
            lemma
        in
            verbs
    }
)

In [14]:
def get_verb(trees: list[dict], verb: Node) -> dict:
    return trees[verb.tree_id].nodes[verb.node_id]

In [ ]:
## TODO:

#
#  + 1. собрать 150 глаголов сов в и 150 несов в ГОТОВО (verbs_data.csv)
#  - 2. категоризовать актанты
#  - 3. тип субъекта и объекта
#  + 4. obl nsubj iobj
#  - 5. наличие отрицания
#  - 6. наречия
#  + 7. все глагольные теги
#  - 8. окна (по 4 слева-справа) или предложения целиком
#  


In [17]:
df

,verb,cyrillic,count,aspect
0,baviti,бавити,9,imp
1,bacati,бацати,6,imp
2,birati,бирати,6,imp
3,biti,бити,5,both
4,blokirati,блокирати,10,imp
...,...,...,...,...
306,formirati,формирати,7,imp
307,funkcionisati,функционисати,7,imp
308,činiti,чинити,24,imp
309,širiti,ширити,5,imp


'imp'

In [11]:
tab = "	"

In [13]:
with open("./data/forms2sents.csv") as file:
    for string in file:
        c = string.count(tab)
        if c != 11:
            print(c)

In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("./data/forms2sents.csv", sep="	", "")

In [17]:
df

,Unnamed: 0,lemma,word,aspect,disambig,feats,nsubj,obj,iobj,obl,rel,text
0,3011,bacati,baca,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,NaN,NaN,NaN,"vodu,NOUN,Case=Loc|Gender=Fem|Number=Sing",conj,"Otpad iz šest priobalnih zemalja, kao i država..."
1,4147,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"razgovori,NOUN,Case=Nom|Gender=Masc|Number=Plur","senku,NOUN,Case=Acc|Gender=Fem|Number=Sing",NaN,"ekonomiju,NOUN,Case=Acc|Gender=Fem|Number=Sing",root,Blokirani razgovori sa MMF bacaju senku na nek...
2,3014,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,NaN,"mulj,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Nu...",NaN,"km,NOUN,_",advcl,"Umesto da bacaju mulj 25-30 km prema pučini, š..."
3,771,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"Srbi,PROPN,Case=Nom|Gender=Masc|Number=Plur","kamenje,NOUN,Case=Acc|Gender=Neut|Number=Sing",NaN,"vojnike,NOUN,Case=Acc|Gender=Masc|Number=Plur;...",root,Kosovski Srbi bacaju kamenje na vojnike KFOR-a...
4,2518,bacati,bacala,imp,imp,Gender=Fem|Number=Sing|Tense=Past|VerbForm=Par...,"publika,NOUN,Case=Nom|Gender=Fem|Number=Sing","flaše,NOUN,Case=Acc|Gender=Fem|Number=Plur",NaN,"večeri,NOUN,Case=Gen|Gender=Neut|Number=Sing;b...",root,"Prve večeri, publika je bacala flaše na bosans..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4564,150,živeti,živi,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,"većina,NOUN,Case=Nom|Gender=Fem|Number=Sing",NaN,NaN,NaN,acl,"Po mom mišljenju, ključno pitanje je način na ..."
4565,4270,živeti,živi,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,"koji,DET,Case=Nom|Gender=Masc|Number=Sing|Pron...",NaN,NaN,"Hancestiju,PROPN,Case=Loc|Gender=Masc|Number=Sing",acl,"Više neću da ih slušam, kaže 42-godišnji inžen..."
4566,1505,živeti,živimo,imp,imp,Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbF...,NaN,NaN,NaN,"njima,PRON,Case=Ins|Number=Plur|Person=3|PronT...",xcomp,"Ne možemo više da živimo sa njima""."
4567,4678,živeti,živimo,imp,imp,Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbF...,NaN,NaN,NaN,"kojoj,DET,Case=Loc|Gender=Fem|Number=Sing|Pron...",acl,"Postavlja se nezaobilazno pitanje, kako se izv..."


In [8]:
for t in df["text"].values:
    if "|" in t:
        print(t)

On nikada nije isticao jedan narod, nego se borio za zemlju, za BiH.
4803	boriti	borila	imp	imp	Gender=Fem|Number=Sing|Tense=Past|VerbForm=Part|Voice=Act	koja,DET,Case=Nom|Gender=Fem|Number=Sing|PronType=Int,Rel			kulisa,NOUN,Case=Gen|Gender=Fem|Number=Plur	acl	Čitava ta „duboka država” (londonski Ekonomist tvrdi da izraz izvorno potiče iz Turske, gde „derin devlet” označava grupu penzionisanih generala koja se iza kulisa borila da svim sredstvima očuva sekularnu državu i porazi komunizam, ne prezajući ni od kontakata sa gangsterima) danas radi na tome da iz Bele kuće istera 45. američkog predsednika koji je izabran suprotno volji elite - političke, vojne, medijske, svake moguće vašingtonske elite, uključujući tu i stranku čiji je kandidat bio.
4814	boriti	bori	imp	imp	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin				vlasti,NOUN,Case=Gen|Gender=Fem|Number=Sing	conj	Vratio se makartizam, samo su progonitelji sada iz liberalne elite i oni kontrolišu frakciju u državi koja se otel

In [6]:
df.head(3)

,lemma,word,aspect,disambig,feats,nsubj,obj,iobj,obl,rel,text
3013,bacati,bacali,imp,imp,Gender=Masc|Number=Plur|Tense=Past|VerbForm=Pa...,"šlepovi,NOUN,Case=Nom|Gender=Masc|Number=Plur","ga,PRON,Case=Acc|Gender=Masc|Number=Sing|Perso...",NaN,"obale,NOUN,Case=Gen|Gender=Fem|Number=Sing",root,"Umesto da bacaju mulj 25-30 km prema pučini, š..."
4147,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"razgovori,NOUN,Case=Nom|Gender=Masc|Number=Plur","senku,NOUN,Case=Acc|Gender=Fem|Number=Sing",NaN,"ekonomiju,NOUN,Case=Acc|Gender=Fem|Number=Sing",root,Blokirani razgovori sa MMF bacaju senku na nek...
2518,bacati,bacala,imp,imp,Gender=Fem|Number=Sing|Tense=Past|VerbForm=Par...,"publika,NOUN,Case=Nom|Gender=Fem|Number=Sing","flaše,NOUN,Case=Acc|Gender=Fem|Number=Plur",NaN,"večeri,NOUN,Case=Gen|Gender=Neut|Number=Sing;b...",root,"Prve večeri, publika je bacala flaše na bosans..."


In [30]:
df.sort_values(by=("lemma", "word"))

KeyError: ('lemma', 'word')

In [8]:
df[["word", "text"]].head(3)

,word,text
3013,bacali,"Umesto da bacaju mulj 25-30 km prema pučini, š..."
4147,bacaju,Blokirani razgovori sa MMF bacaju senku na nek...
2518,bacala,"Prve večeri, publika je bacala flaše na bosans..."


In [ ]:
    # *   ја *нисам* (ja *nisam*) – я не есть (я не являюсь, я не)
    # *   ти *ниси* (ti *nisi*) – ты не есть (ты не являешься, ты не)
    # *   он/она/оно *није* (on/ona/ono *nije*) – он/она/оно не есть (он/она/оно не является, он/она/оно не)
    # *   ми *нисмо* (mi *nismo*) – мы не есть (мы не являемся, мы не)
    # *   ви *нисте* (vi *niste*) – вы не есть (вы не являетесь, вы не)
    # *   они *нису* (oni *nisu*) – они не есть (они не являются, они не)


In [26]:
ff = []

for i, row in df[["word", "text"]].iterrows():
    tokens = row["text"].split()
    c = tokens.count(row["word"])
    if c > 1:
        print(c, row["word"])
        ff.append(i)

len(ff)
    # print(row["word"])

2 da
2 da
2 daje
2 daje
5 dobila
3 dodao
2 donose
2 donose
5 došlo
2 funkcioniše
2 glasati
2 imam
2 imam
2 imamo
3 imamo
5 imamo
2 ima
3 izjavio
2 iznosio
2 iznosio
5 kaže
3 koristi
2 mora
2 morati
2 moraju
2 mora
3 morati
4 moraju
3 moramo
3 mogu
3 moći
3 može
2 može
2 mogla
2 može
2 može
2 mogla
3 Nadamo
2 nema
14 nema
2 nema
5 odigrali
5 odlučio
3 odvija
2 olakšalo
2 olakšalo
3 omogućavaju
7 ostati
6 ostati
3 ode
5 očekuje
3 podstiču
4 pokušati
2 ponoviti
6 ponuditi
5 postane
3 postigne
2 postoji
3 Postoji
2 postoji
7 postoji
3 poštuje
5 prati
2 predstavlja
2 predstavlja
5 predstavljaju
3 prevazići
2 pridružiti
8 priznati
8 promeniti
4 putovati
5 radi
2 radi
3 razmotri
5 razumeti
7 saopštila
5 sarađuje
2 sastati
2 smanjiti
2 smanjiti
2 sprečiti
2 sprečiti
3 trajati
3 treba
3 treba
3 treba
2 trebalo
5 treba
2 tvrde
5 uđu
6 učestvovati
3 vidimo
4 vidimo
3 vrše
3 vrše
3 znači
5 znači
3 želi
3 žele
2 žive
2 žive


101

In [27]:
ff

[642,
 3891,
 4734,
 4737,
 2868,
 780,
 4428,
 4427,
 3505,
 2342,
 1324,
 3795,
 3794,
 777,
 919,
 51,
 1997,
 2343,
 3068,
 3069,
 3461,
 1338,
 1862,
 1196,
 3575,
 1864,
 1187,
 3573,
 129,
 4445,
 1996,
 3648,
 3453,
 4167,
 236,
 239,
 4164,
 3936,
 3164,
 1998,
 3163,
 3511,
 778,
 3667,
 1963,
 1962,
 1994,
 1391,
 3937,
 3314,
 2142,
 2732,
 4437,
 2509,
 507,
 4435,
 50,
 4639,
 573,
 4640,
 3666,
 2055,
 3829,
 4719,
 4718,
 3961,
 921,
 2755,
 2753,
 636,
 1975,
 3460,
 4988,
 1189,
 2251,
 2752,
 95,
 1902,
 4151,
 4153,
 1388,
 1389,
 3576,
 4761,
 4757,
 4759,
 3462,
 2869,
 1,
 1986,
 3660,
 2849,
 2922,
 3326,
 3730,
 1995,
 2754,
 3176,
 1985,
 3864,
 3866]

In [33]:
df.sort_values(["lemma", "word"]).to_csv("./data/forms2sents.csv", sep="\t")

In [18]:
import pandas as pd

In [20]:
import csv

In [21]:
quoting=csv.QUOTE_NONE

In [ ]:
df = pd.read_csv("./data/forms2sents.csv", sep="	", quoting=csv.QUOTE_NONE)

In [23]:
df

,Unnamed: 0,lemma,word,aspect,disambig,feats,nsubj,obj,iobj,obl,rel,text
0,3011,bacati,baca,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,NaN,NaN,NaN,"vodu,NOUN,Case=Loc|Gender=Fem|Number=Sing",conj,"Otpad iz šest priobalnih zemalja, kao i država..."
1,4147,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"razgovori,NOUN,Case=Nom|Gender=Masc|Number=Plur","senku,NOUN,Case=Acc|Gender=Fem|Number=Sing",NaN,"ekonomiju,NOUN,Case=Acc|Gender=Fem|Number=Sing",root,Blokirani razgovori sa MMF bacaju senku na nek...
2,3014,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,NaN,"mulj,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Nu...",NaN,"km,NOUN,_",advcl,"Umesto da bacaju mulj 25-30 km prema pučini, š..."
3,771,bacati,bacaju,imp,imp,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbF...,"Srbi,PROPN,Case=Nom|Gender=Masc|Number=Plur","kamenje,NOUN,Case=Acc|Gender=Neut|Number=Sing",NaN,"vojnike,NOUN,Case=Acc|Gender=Masc|Number=Plur;...",root,Kosovski Srbi bacaju kamenje na vojnike KFOR-a...
4,2518,bacati,bacala,imp,imp,Gender=Fem|Number=Sing|Tense=Past|VerbForm=Par...,"publika,NOUN,Case=Nom|Gender=Fem|Number=Sing","flaše,NOUN,Case=Acc|Gender=Fem|Number=Plur",NaN,"večeri,NOUN,Case=Gen|Gender=Neut|Number=Sing;b...",root,"""Prve večeri, publika je bacala flaše na bosan..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5003,150,živeti,živi,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,"većina,NOUN,Case=Nom|Gender=Fem|Number=Sing",NaN,NaN,NaN,acl,"Po mom mišljenju, ključno pitanje je način na ..."
5004,4270,živeti,živi,imp,imp,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,"koji,DET,Case=Nom|Gender=Masc|Number=Sing|Pron...",NaN,NaN,"Hancestiju,PROPN,Case=Loc|Gender=Masc|Number=Sing",acl,"Više neću da ih slušam, kaže 42-godišnji inžen..."
5005,1505,živeti,živimo,imp,imp,Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbF...,NaN,NaN,NaN,"njima,PRON,Case=Ins|Number=Plur|Person=3|PronT...",xcomp,"""Ne možemo više da živimo sa njima""""."""
5006,4678,živeti,živimo,imp,imp,Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbF...,NaN,NaN,NaN,"kojoj,DET,Case=Loc|Gender=Fem|Number=Sing|Pron...",acl,"Postavlja se nezaobilazno pitanje, kako se izv..."


In [25]:
df.to_csv("./data/forms2sents222.csv", sep="	", quoting=csv.QUOTE_NONE, index=None)